# उत्पादनामध्ये AutoGen एजंट्स: निरीक्षणक्षमता आणि मूल्यमापन

या ट्यूटोरियलमध्ये, आपण **[Autogen agents](https://github.com/microsoft/autogen)** च्या अंतर्गत टप्प्यांचे (ट्रेसेस) निरीक्षण कसे करायचे आणि [Langfuse](https://langfuse.com) चा वापर करून त्याच्या कार्यक्षमतेचे मूल्यमापन कसे करायचे ते शिकणार आहोत.

हा मार्गदर्शक **ऑनलाइन** आणि **ऑफलाइन** मूल्यमापन मेट्रिक्स कव्हर करतो, ज्याचा वापर टीम्स एजंट्सना वेगाने आणि विश्वासार्हतेने उत्पादनात आणण्यासाठी करतात.

**एआय एजंट मूल्यमापन का महत्त्वाचे आहे:**
- जेव्हा कार्य अयशस्वी होते किंवा अपूर्ण परिणाम देतात तेव्हा समस्या शोधणे
- रिअल-टाइममध्ये खर्च आणि कार्यक्षमता निरीक्षण करणे
- सतत अभिप्रायाद्वारे विश्वासार्हता आणि सुरक्षितता सुधारणे


## चरण 1: पर्यावरणीय चल (Environment Variables) सेट करा

[Langfuse Cloud](https://cloud.langfuse.com/) साठी साइन अप करून किंवा [स्वतः होस्टिंग Langfuse](https://langfuse.com/self-hosting) वापरून तुमची Langfuse API कीज मिळवा.

_**टीप:** स्वतः होस्ट करणारे [Terraform modules](https://langfuse.com/self-hosting/azure) वापरून Azure वर Langfuse तैनात करू शकतात. पर्यायाने, तुम्ही [Helm chart](https://langfuse.com/self-hosting/kubernetes-helm) वापरून Kubernetes वर Langfuse तैनात करू शकता._


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

पर्यावरणीय चलांचे मूल्य सेट केल्यानंतर, आपण आता Langfuse क्लायंट प्रारंभ करू शकतो. `get_client()` पर्यावरणीय चलांमध्ये दिलेल्या ओळखपत्रांचा वापर करून Langfuse क्लायंट प्रारंभ करते.


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## चरण 2: OpenLit साधनसंपत्ती प्रारंभ करा

आता आपण [OpenLit](https://github.com/openlit/openlit) साधनसंपत्ती प्रारंभ करतो. OpenLit आपोआप AutoGen ऑपरेशन्स कॅप्चर करते आणि OpenTelemetry (OTel) स्पॅन्स Langfuse मध्ये निर्यात करते.


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## चरण ३: तुमचा एजंट चालवा

आता आपण आमच्या उपकरणांची चाचणी करण्यासाठी एक मल्टी-टर्न एजंट सेट अप करतो.


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### ट्रेस संरचना

Langfuse एक **ट्रेस** नोंदवतो ज्यामध्ये **स्पॅन्स** असतात, जे तुमच्या एजंटच्या लॉजिकमधील प्रत्येक टप्प्याचे प्रतिनिधित्व करतात. येथे, ट्रेसमध्ये एकूण एजंट रन आणि खालील उप-टप्पे समाविष्ट आहेत:
- मील प्लॅनर एजंट
- न्यूट्रिशनिस्ट एजंट्स

तुम्ही यांचे निरीक्षण करून अचूकपणे पाहू शकता की वेळ कुठे खर्च होतो, किती टोकन्स वापरले जातात, इत्यादी:

![Langfuse मधील ट्रेस ट्री](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[ट्रेससाठी लिंक](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## ऑनलाइन मूल्यांकन

ऑनलाइन मूल्यांकन म्हणजे एजंटचे मूल्यांकन थेट, वास्तविक जगातील वातावरणात करणे, म्हणजे उत्पादनात प्रत्यक्ष वापरादरम्यान. यामध्ये एजंटच्या वास्तविक वापरकर्त्यांसोबतच्या संवादांवर सतत नजर ठेवणे आणि परिणामांचे विश्लेषण करणे समाविष्ट आहे.

### उत्पादनात ट्रॅक करण्यासाठी सामान्य मेट्रिक्स

1. **खर्च** — उपकरणे टोकन वापर ट्रॅक करतात, ज्याला तुम्ही प्रत्येक टोकनसाठी किंमत देऊन अंदाजे खर्चात रूपांतरित करू शकता.
2. **विलंबता** — प्रत्येक चरण पूर्ण होण्यासाठी किंवा संपूर्ण प्रक्रिया पूर्ण होण्यासाठी लागणारा वेळ निरीक्षण करा.
3. **वापरकर्ता अभिप्राय** — वापरकर्ते थेट अभिप्राय देऊ शकतात (थंब्स अप/डाउन) ज्यामुळे एजंट सुधारण्यात किंवा दुरुस्त करण्यात मदत होते.
4. **LLM-जज म्हणून** — तुमच्या एजंटच्या आउटपुटचे जवळपास वास्तविक वेळेत मूल्यांकन करण्यासाठी स्वतंत्र LLM वापरा (उदाहरणार्थ, विषारीपणा किंवा अचूकता तपासणे).

खाली, या मेट्रिक्सचे उदाहरणे दाखवले आहेत.


#### 1. खर्च

खालील स्क्रीनशॉटमध्ये `gpt-4o-mini` कॉल्ससाठीचा वापर दाखवलेला आहे. हे महागड्या टप्प्यांची ओळख पटवण्यासाठी आणि तुमच्या एजंटला ऑप्टिमाइझ करण्यासाठी उपयुक्त आहे.

![खर्च](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png)

_[ट्रेससाठी लिंक](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. विलंब

आपण प्रत्येक टप्पा पूर्ण होण्यासाठी किती वेळ लागला हे देखील पाहू शकतो. खालील उदाहरणात, संपूर्ण प्रक्रिया पूर्ण होण्यासाठी सुमारे 3 सेकंद लागले, ज्याला तुम्ही टप्प्यांमध्ये विभागू शकता. यामुळे तुम्हाला अडथळे ओळखण्यात आणि तुमच्या एजंटला ऑप्टिमाइझ करण्यात मदत होते.

![Latency](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[ट्रेससाठी दुवा](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. वापरकर्ता अभिप्राय

जर तुमचा एजंट वापरकर्ता इंटरफेसमध्ये समाविष्ट असेल, तर तुम्ही थेट वापरकर्त्याचा अभिप्राय नोंदवू शकता (जसे की चॅट UI मध्ये अंगठा वर/खाली).


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

वापरकर्ता अभिप्राय नंतर Langfuse मध्ये नोंदवला जातो:

![वापरकर्ता अभिप्राय Langfuse मध्ये नोंदवला जात आहे](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. स्वयंचलित LLM-as-a-Judge स्कोरिंग

LLM-as-a-Judge हा तुमच्या एजंटच्या आउटपुटचे स्वयंचलितपणे मूल्यांकन करण्याचा आणखी एक मार्ग आहे. तुम्ही आउटपुटची अचूकता, विषारीपणा, शैली किंवा तुम्हाला महत्त्वाच्या असलेल्या इतर निकषांचे मूल्यांकन करण्यासाठी स्वतंत्र LLM कॉल सेट करू शकता.

**कार्यप्रवाह**:
1. तुम्ही एक **मूल्यांकन टेम्पलेट** परिभाषित करता, उदा., "पाठ विषारी आहे का ते तपासा."
2. तुम्ही एक मॉडेल सेट करता जे न्यायाधीश-मॉडेल म्हणून वापरले जाते; या प्रकरणात `gpt-4o-mini` जो Azure द्वारे क्वेरी केला जातो.
2. प्रत्येक वेळी तुमचा एजंट आउटपुट तयार करतो, तुम्ही ते आउटपुट टेम्पलेटसह "न्यायाधीश" LLM कडे पाठवता.
3. न्यायाधीश LLM रेटिंग किंवा लेबलसह प्रतिसाद देतो, जे तुम्ही तुमच्या निरीक्षण साधनात नोंदवता.

Langfuse मधील उदाहरण:

![LLM-as-a-Judge Evaluator](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

तुम्ही पाहू शकता की या उदाहरणाचे उत्तर "विषारी नाही" असे ठरवले गेले आहे.

![LLM-as-a-Judge मूल्यांकन स्कोअर](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### 5. निरीक्षणक्षमतेचे मेट्रिक्स आढावा

हे सर्व मेट्रिक्स डॅशबोर्डमध्ये एकत्रितपणे पाहता येतात. यामुळे तुम्हाला अनेक सत्रांमध्ये तुमचा एजंट कसा कार्य करतो हे जलदपणे पाहता येते आणि वेळोवेळी गुणवत्ता मेट्रिक्स ट्रॅक करण्यास मदत होते.

![निरीक्षणक्षमतेचे मेट्रिक्स आढावा](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## ऑफलाइन मूल्यांकन

ऑनलाइन मूल्यांकन हे त्वरित अभिप्रायासाठी महत्त्वाचे आहे, परंतु तुम्हाला **ऑफलाइन मूल्यांकन** देखील आवश्यक आहे—विकासाच्या आधी किंवा दरम्यान केलेल्या पद्धतशीर तपासण्या. यामुळे उत्पादनात बदल लागू करण्यापूर्वी गुणवत्ता आणि विश्वासार्हता टिकवून ठेवण्यास मदत होते.


### डेटासेट मूल्यांकन

ऑफलाइन मूल्यांकनामध्ये, तुम्ही सहसा हे करता:
1. एक बेंचमार्क डेटासेट असतो (प्रॉम्प्ट आणि अपेक्षित उत्तरांच्या जोड्यांसह)
2. तुमचा एजंट त्या डेटासेटवर चालवता
3. आउटपुट्सची अपेक्षित निकालांशी तुलना करता किंवा अतिरिक्त स्कोअरिंग यंत्रणा वापरता

खाली, आम्ही [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset) सह हा दृष्टिकोन दाखवतो, ज्यामध्ये प्रश्न आणि अपेक्षित उत्तरे समाविष्ट आहेत.


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

पुढे, रन ट्रॅक करण्यासाठी Langfuse मध्ये एक डेटासेट एंटिटी तयार करतो. नंतर, डेटासेटमधील प्रत्येक आयटम प्रणालीमध्ये जोडतो.


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![डेटासेट आयटम्स लँगफ्यूजमध्ये](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### डेटासेटवर एजंट चालवणे

सुरुवातीला, आम्ही एक साधा Autogen एजंट तयार करतो जो Azure OpenAI मॉडेल्सचा वापर करून प्रश्नांची उत्तरे देतो.


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

मग, आपण `my_agent()` ही सहाय्यक फंक्शन परिभाषित करतो.


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

शेवटी, आपण प्रत्येक डेटासेट आयटमवर लूप करतो, एजंट चालवतो, आणि ट्रेसला डेटासेट आयटमशी लिंक करतो. आवश्यक असल्यास, आपण जलद मूल्यांकन गुण देखील जोडू शकतो.


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

आपण वेगवेगळ्या एजंट कॉन्फिगरेशन्ससह हा प्रक्रिया पुन्हा करू शकता, जसे की:
- मॉडेल्स (gpt-4o-mini, gpt-4.1, इ.)
- प्रॉम्प्ट्स
- टूल्स (सर्च विरुद्ध नो सर्च)
- एजंटची गुंतागुंत (मल्टी एजंट विरुद्ध सिंगल एजंट)

यानंतर, त्यांची तुलना Langfuse मध्ये बाजूने बाजूने करा. या उदाहरणात, मी एजंटला 25 डेटासेट प्रश्नांवर 3 वेळा चालवले. प्रत्येक रनसाठी, मी वेगळ्या Azure OpenAI मॉडेलचा वापर केला. आपण पाहू शकता की मोठ्या मॉडेलचा वापर केल्यास योग्य उत्तर दिलेल्या प्रश्नांची संख्या सुधारते (अपेक्षेप्रमाणे). `correct_answer` स्कोअर [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) द्वारे तयार केला जातो, जो डेटासेटमध्ये दिलेल्या नमुना उत्तराच्या आधारे प्रश्नाची योग्यतादर्शकता न्याय करण्यासाठी सेट केला आहे.

![डेटासेट रनचा आढावा](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)
![डेटासेट रन तुलना](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील मूळ दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
